Estrutura de Dados - Pilha e fila
==========================================

Capítulo 10 do livro texto sugerido:
Introduction to Algorithms, Fourth Edition
By Thomas H. Cormen, Charles E. Leiserson, Ronald L. Rivest and Clifford Stein
https://mitpress.mit.edu/9780262046305/introduction-to-algorithms/

Conteúdo
========

Como mostrado na Seção de introdução, pilhas e filas são estruturas implementadas de maneira semelhante.

Sua maior diferente é quanto a ordem de entrada e saída dos dados: First-In Last-Out (FILO) versus First-In First Out (FIFO).

Revisitamos as duas estruturas a seguir:

## Pilhas

In [1]:
class ElementoPilha():
    def __init__(self, valorInicial):
        self.valor = valorInicial
        self.elementoInferior = None

class Pilha():
    def __init__(self):
        self.topo = None

    def empilha(self, valor):
        novoTopo = ElementoPilha(valorInicial=valor)
        if self.topo is not None:
            novoTopo.elementoInferior = self.topo
        self.topo = novoTopo

    def desempilha(self):
        if self.topo is None:
            return None
        topoAntigo = self.topo
        self.topo = topoAntigo.elementoInferior
        return topoAntigo.valor

pilha = Pilha()
print(pilha.desempilha())
pilha.empilha(1)
pilha.empilha(2)
print(pilha.desempilha())
pilha.empilha(3)
print(pilha.desempilha())
print(pilha.desempilha())
print(pilha.desempilha())

None
2
3
1
None


Mas em que aplicações são utilizadas estas estruturas?

Pilhas são utilizadas por programas quando querem chamar uma função.
São empilhados os parâmetros da função a ser chamada (callee), em seguida do endereço de retorno (return address) para a próxima instrução do chamador (caller) após a função. O programa então pula para o endereço da função chamada (callee), quando empilha o endereço de base da pilha EBP do chamador (caller) e então inicia sua execução. Ao fim, reestabelece o endereço de base da pilha desempilhando o valor, e então salta para o endereço de retorno especificado.

### Quadro de pilha de programa C (https://norasandler.com/2018/01/08/Write-a-Compiler-5.html)
![](./03_pilha_e_fila/c_stack_frame.png)

De maneira semelhante, o Python tem seus próprios quadros. Estes quadros também contém argumentos, variáveis definidas dentro de uma função (locais),
e um endereço de retorno (representado por uma referência ao quadro anterior na pilha).

### Quadro de pilha de programa Python (https://towardsdatascience.com/python-stack-frames-and-tail-call-optimization-4d0ea55b0542)
![](./03_pilha_e_fila/python_stack_frame.png)

Nós podemos acessar e visualizar os conteúdos destes quadros e da pilha. Vejamos dois exemplos.

In [2]:
import sys

def funcao1(argumento1, argumento2):
    print("Funcao1:")
    frame = sys._getframe()
    print("Variáveis locais:", frame.f_locals)
    print("Quadro de retorno:", frame.f_back)

funcao1(1, 2)

Funcao1:
Variáveis locais: {'argumento1': 1, 'argumento2': 2, 'frame': <frame at 0x7fc0988979a0, file '/tmp/ipykernel_9014/831056386.py', line 6, code funcao1>}
Quadro de retorno: <frame at 0x7fc0988a17b0, file '/tmp/ipykernel_9014/831056386.py', line 9, code <module>>


Agora vejamos uma outra função, com sua pilha de quadros de execução completa:

In [3]:
import traceback

def funcao2(argumentoA, argumentoB):
    print("Funcao2:")
    frame = sys._getframe()
    print("Variáveis locais:", frame.f_locals)
    print("Quadro de retorno:", frame.f_back)
    print("Pilha de quadros:")
    traceback.print_stack(file=sys.stdout)

funcao2("1", "2")

Funcao2:
Variáveis locais: {'argumentoA': '1', 'argumentoB': '2', 'frame': <frame at 0x7fc098897d00, file '/tmp/ipykernel_9014/3121221515.py', line 6, code funcao2>}
Quadro de retorno: <frame at 0x7fc0988a1940, file '/tmp/ipykernel_9014/3121221515.py', line 11, code <module>>
Pilha de quadros:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 982, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 725, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 215, in start
    self.asyncio_loop.run_forever

Note que essa pilha enorme não é normal, e sim devido ao código ter sido executado pelo notebook Jupyter.
Se o mesmo programa é executado diretamente do interpretador CPython, obtemos o seguinte resultado impresso.


```
/mnt/dev/tools/source/ed_2022_2/venv/bin/python /mnt/dev/tools/source/ed_2022_2/test.py
Funcao2:
Variáveis locais: {'argumentoA': '1', 'argumentoB': '2', 'frame': <frame at 0x7fc3cc3dcfc0, file '/mnt/dev/tools/source/ed_2022_2/test.py', line 7, code funcao2>}
Quadro de retorno: <frame at 0x7fc3cc349e40, file '/mnt/dev/tools/source/ed_2022_2/test.py', line 13, code <module>>
Pilha de quadros:
  File "/mnt/dev/tools/source/ed_2022_2/test.py", line 13, in <module>
    funcao2("1", "2")
  File "/mnt/dev/tools/source/ed_2022_2/test.py", line 10, in funcao2
    traceback.print_stack(file=sys.stdout)
```

É realmente fantástico. Mas sabem onde mais as pilhas são comumente utilizadas? Máquinas de pilhas.

O que são máquinas de pilhas? São máquinas que funcionam empilhando parâmetros e desempilhando conforme operadores são chamados.

Como ninguém melhor que os próprios operandos para saber quantos parâmetros recebem, este tipo de implementação é bastante comum.

E surpresa, o Python é um interpretador que utiliza máquina de pilhas.

A especificação dos operandos está disponível em https://docs.python.org/3/library/dis.html.

In [4]:
from dis import dis

def fatorial(n):
    if n <= 1:
        return 1
    else:
        return n*fatorial(n-1)

dis(fatorial)

  4           0 LOAD_FAST                0 (n)
              2 LOAD_CONST               1 (1)
              4 COMPARE_OP               1 (<=)
              6 POP_JUMP_IF_FALSE        6 (to 12)

  5           8 LOAD_CONST               1 (1)
             10 RETURN_VALUE

  7     >>   12 LOAD_FAST                0 (n)
             14 LOAD_GLOBAL              0 (fatorial)
             16 LOAD_FAST                0 (n)
             18 LOAD_CONST               1 (1)
             20 BINARY_SUBTRACT
             22 CALL_FUNCTION            1
             24 BINARY_MULTIPLY
             26 RETURN_VALUE


Existem diversos detalhes de implementação que são complexos de se entender, portanto utilizemos uma implementação mais simples de máquinas de pilha.

In [5]:
class MaquinaDePilha:
    def __init__(self):
        # Cria pilha de operandos/operadores
        self.pilhaOperandos = Pilha()
        # Define funções que implementam as
        # operações suportadas pela máquina de pilha
        def soma(x,y):
            return x+y
        def subt(x,y):
            return x-y
        def mult(x,y):
            return x*y
        def divi(x,y):
            return x/y
        # Cria um dicionário mapeando
        # operadores em operações
        self.operadores = {"+": soma,
                           "-": subt,
                           "*": mult,
                           "/": divi
        }

    # Define o método que empilha operandos
    def empilhaOperando(self, operando):
        # Caso de fato seja operando, empilha seu valor na pilha
        if operando not in self.operadores:
            print("Empilha:", operando)
            self.pilhaOperandos.empilha(operando)
        # Caso seja um operador, desempilhe 2 parâmetros,
        # execute a operação e empilhe o resultado
        else:
            print(f"Operador: {operando}")
            operandoA = self.pilhaOperandos.desempilha()
            print(f"Desempilha: {operandoA}")
            operandoB = self.pilhaOperandos.desempilha()
            print(f"Desempilha: {operandoB}")
            resultado = self.operadores[operando](operandoA, operandoB)
            print(f"Executa: {operandoA} {operando} {operandoB} = {resultado}")
            self.pilhaOperandos.empilha(resultado)
            print(f"Empilha: {resultado}")

    # Desempilha um operando
    def desempilhaOperando(self):
        valor = self.pilhaOperandos.desempilha()
        print(f"Desempilha: {valor}")
        return valor

Podemos agora testar o funcionamento desta máquina de pilha.

In [6]:
maquinaDePilha = MaquinaDePilha()

maquinaDePilha.empilhaOperando(1)
maquinaDePilha.empilhaOperando(2)
maquinaDePilha.empilhaOperando("+")
maquinaDePilha.empilhaOperando(3)
maquinaDePilha.empilhaOperando("-")
resultado = maquinaDePilha.desempilhaOperando()
print(resultado)

Empilha: 1
Empilha: 2
Operador: +
Desempilha: 2
Desempilha: 1
Executa: 2 + 1 = 3
Empilha: 3
Empilha: 3
Operador: -
Desempilha: 3
Desempilha: 3
Executa: 3 - 3 = 0
Empilha: 0
Desempilha: 0
0


### Filas

Filas são bastante comuns devido a característica First-In First-Out, especialmente quando tentamos gerenciar recursos e atividades.

Se já aguardou sua vez em uma fila, sabe exatamente do que estamos falando.

In [7]:
class ElementoFila():
    def __init__(self, valorInicial):
        self.valor = valorInicial
        self.anterior = None

class Fila():
    def __init__(self):
        self.inicioFila = None
        self.fimFila = None

    def enfileira(self, valor):
        novoFimFila = ElementoFila(valorInicial=valor)
        if self.fimFila is not None:
            self.fimFila.anterior = novoFimFila
        self.fimFila = novoFimFila
        if self.inicioFila is None:
            self.inicioFila = self.fimFila

    def desenfileira(self):
        if self.inicioFila is None:
            self.fimFila = None
            return None
        antigoInicioFila = self.inicioFila
        self.inicioFila = antigoInicioFila.anterior
        return antigoInicioFila.valor

fila = Fila()
print(fila.desenfileira())
fila.enfileira(1)
fila.enfileira(2)
print(fila.desenfileira())
fila.enfileira(3)
print(fila.desenfileira())
print(fila.desenfileira())
print(fila.desenfileira())

None
1
2
3
None


Mas onde podemos usar estas filas?

Uma aplicação recorrente é uma fila de trabalho, usada por produtores e consumidores.

Produtores produzem trabalho, ou ordens de trabalho, e consumidores consomem os frutos do trabalho, ou executam as ordens de trabalho.

In [8]:
import random

# Mesmos 10000 inteiros aleatórios
inteirosAleatorios = [random.randint(0, 1000) for i in range(1000)]

# Bubble sort visto anteriormente, agora consome itens da fila de trabalho
# e devolve a resposta na fila de respostas
def bubbleSort(filaTrabalho, filaResposta):
    # Continua trabalhando enquanto ouver trabalho na fila
    while not filaTrabalho.empty():
        # Remove trabalho da fila de trabalho
        lista = filaTrabalho.get()
        # Marca trabalho como feito para permitir que o
        # outro lado da fila coloque um novo trabalho nesta
        filaTrabalho.task_done()
        # Organiza a lista
        for i in range(len(lista)):
            for j in range(i+1, len(lista)):
                if lista[i] > lista[j]:
                    temp = lista[i]
                    lista[i] = lista[j]
                    lista[j] = temp
        # Guarda lista organizada na fila de respostas
        filaResposta.put_nowait(lista)
    return None

# Criamos as filas de trabalho e repostas, com capacidade para 100 trabalhos cada
from queue import Queue
filaDeTrabalho = Queue(maxsize=10)
filaDeRepostas = Queue(maxsize=10)

# Quebramos os 10000 inteiros aleatórios em 10 trabalhos e enfileiramos o trabalho
for i in range(10):
    filaDeTrabalho.put_nowait(inteirosAleatorios[i*100:(i+1)*100])

# Lançamos 4 threads para consumir a fila de trabalho e organizar cada sublista com inteiros
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=4) as pool:
    pool.map(bubbleSort, [filaDeTrabalho]*4, [filaDeRepostas]*4)

# Lêmos as respostas da fila de respostas
listasParciaisOrdenadas = []
while not filaDeRepostas.empty():
    # Lê lista parcial ordenada da fila de respostas
    listasParciaisOrdenadas.append(filaDeRepostas.get())
    # Marca tarefa como pronta para que outra resposta seja enfileirada
    filaDeRepostas.task_done()
    # Imprime começo das listas parciais para ver
    # que estão ordenadas individualmente, mas não globalmente
    print("Lista parcialmente ordenada:", listasParciaisOrdenadas[-1][:10])

Lista parcialmente ordenada: [5, 16, 33, 54, 79, 85, 91, 96, 113, 113]
Lista parcialmente ordenada: [24, 26, 36, 85, 89, 98, 106, 136, 169, 172]
Lista parcialmente ordenada: [2, 4, 6, 14, 30, 39, 42, 56, 64, 76]
Lista parcialmente ordenada: [8, 25, 33, 36, 38, 38, 42, 55, 66, 74]
Lista parcialmente ordenada: [4, 15, 17, 27, 31, 34, 54, 63, 65, 95]
Lista parcialmente ordenada: [25, 33, 40, 54, 66, 79, 90, 90, 91, 105]
Lista parcialmente ordenada: [2, 2, 4, 5, 9, 31, 37, 41, 95, 102]
Lista parcialmente ordenada: [10, 13, 25, 28, 37, 61, 70, 72, 75, 89]
Lista parcialmente ordenada: [10, 10, 44, 59, 79, 82, 88, 88, 89, 92]
Lista parcialmente ordenada: [1, 7, 9, 16, 28, 31, 45, 46, 63, 75]


Ainda precisamos juntar as listas parciais ordenadas. Isto tipicamente é feito sequencialmente.

In [9]:
# Junta listas parcialmente ordenadas
def k_way_merge(listas_parcialmente_ordenadas):
    resultadoFinal = []

    while True:
        num_listas = len(listas_parcialmente_ordenadas)
        # Remove listas parciais vazias
        for indice_lista in map(lambda x: num_listas-x-1, range(num_listas)):
            if len(listas_parcialmente_ordenadas[indice_lista]) == 0:
                listas_parcialmente_ordenadas.pop(indice_lista)

        # Se não houverem mais listas parciais para juntar, retorne a lista ordenada
        if len(listas_parcialmente_ordenadas) == 0:
            break

        # Capture a lista dos primeiros itens de todas as listas parciais
        lista = list(map(lambda x: x[0], listas_parcialmente_ordenadas))

        # Remova o valor mínimo de todas as listas em que ocorre
        min_val = min(lista)
        while True:
            try:
                # Procura em que lista o valor mínimo aparece
                indice_lista = lista.index(min_val)
            except ValueError:
                # Quando o valor mínimo não ocorrer mais, procura o próximo valor mínimo
                break
            # Remove o valor mínimo encontrado na frente da lista indice_lista
            resultadoFinal.append(listas_parcialmente_ordenadas[indice_lista][0])
            listas_parcialmente_ordenadas[indice_lista].pop(0)

            # Remove o índice para evitar tentativa de remover duas vezes o mesmo valor
            lista[indice_lista] = None
    return resultadoFinal

print("Lista totalmente ordenada:", k_way_merge(listasParciaisOrdenadas)[:140])

Lista totalmente ordenada: [1, 2, 2, 2, 4, 4, 4, 5, 5, 6, 7, 8, 9, 9, 10, 10, 10, 13, 14, 15, 16, 16, 17, 24, 25, 25, 25, 26, 27, 28, 28, 30, 31, 31, 31, 33, 33, 33, 34, 36, 36, 37, 37, 38, 38, 39, 40, 41, 42, 42, 44, 45, 46, 54, 54, 54, 55, 56, 59, 61, 63, 63, 64, 65, 66, 66, 70, 72, 74, 75, 75, 76, 79, 79, 79, 82, 85, 85, 88, 88, 89, 89, 89, 89, 90, 90, 91, 91, 92, 92, 92, 92, 95, 95, 95, 96, 98, 98, 99, 102, 102, 102, 104, 105, 105, 105, 106, 106, 106, 108, 113, 113, 113, 115, 116, 117, 117, 118, 118, 121, 123, 123, 124, 125, 126, 126, 126, 127, 130, 131, 133, 134, 134, 134, 135, 136, 136, 137, 137, 137]
